In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.impute import SimpleImputer
lr = 0.3
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import confusion_matrix   
import math

In [2]:
import pandas as pd
# reading csv files
df =  pd.read_csv('heart.dat', sep=" ", header=None)

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,1
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,1
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52.0,1.0,3.0,172.0,199.0,1.0,0.0,162.0,0.0,0.5,1.0,0.0,7.0,1
266,44.0,1.0,2.0,120.0,263.0,0.0,0.0,173.0,0.0,0.0,1.0,0.0,7.0,1
267,56.0,0.0,2.0,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,3.0,1
268,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,1


In [4]:
column_name = ['age', 'sex', 'chest pain', 'resting blood pressure', 'serum cholestoral', 'fasting blood sugar', 'resting electrocardiographic', 'maximum heart rate achieved',' exercise induced angina', 'oldpeak',' the slope of the peak exercise ST segment', 'number of major vessels', 'tha', 'label']

In [6]:
df.columns = column_name
df = df.fillna(0)
X = df.iloc[:, :-1]

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
trans = StandardScaler()
X = trans.fit_transform(X)

In [10]:
label = df.iloc[:, -1]
label = label.map({1:0, 2:1})
label = label.map({0:0, 1:1, 2:1, 3:1, 4:1})
label = np.array(label)

## Feature important

In [17]:
from sklearn.ensemble import RandomForestClassifier
X_Imputed = X
feature_names = [f'feature {i}' for i in range(X_Imputed.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_Imputed, label)
importances = forest.feature_importances_

In [18]:
df_imprtance = pd.DataFrame(importances)
index_top_n_Feature = df_imprtance.sort_values(by=df_imprtance.columns[0],ascending=False).iloc[:12,].index
X_Imputed_df = pd.DataFrame(X_Imputed)
X_Imputed_df = X_Imputed_df.iloc[:,index_top_n_Feature.values]

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_Imputed_df, label, test_size = 0.3, random_state = 0)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow

def training(X, y, iteration, epoch):
    #iteration = iteration+1
    # define the keras model
    model_path = 'DL_model_Bagging_Numeric/model_'+str(iteration)+'.h5'
    model = Sequential()
    model.add(Dense(248, input_dim=X.shape[-1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2,  activation = "softmax"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    model.fit(X, y, 
              epochs=epoch,
              batch_size=2000,
              validation_split=0.25,
              verbose=0,
              #class_weight=class_weights, 
              callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                    tensorflow.keras.callbacks.ModelCheckpoint(model_path,
                                                                               monitor='val_loss',
                                                                               save_best_only=True,
                                                                               mode='min',
                                                                               verbose=0)])
    print("======================= Done for iteration ", iteration)

## Device data into bag

In [27]:
def lr_scheduler(epoch, lr):
    if epoch % 50 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

In [28]:
from tensorflow.keras.models import load_model

def model_Predict(X):
    RM_predicted =[]
    for i in range(0, num_Interation):
        model = load_model('DL_model_Bagging_Numeric/model_'+str(i)+".h5")
        RM_predicted.append(model.predict(X))        
    return RM_predicted

In [29]:
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

## Running on Multiple

In [ ]:
### from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import tensorflow
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

import timeit
import tracemalloc
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical



# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


max_auc = 0.88
for num_Interation in range(1, 50):
    rsFile = "Result_OverIteration_Bagging_Numeric/NumberOfBag_"+str(num_Interation)+".csv"
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for epoch in range(10, 500, 10):
    
        print("working on num_Interation: " + str(num_Interation) + "_with_epoch_"+str(epoch))
        
        X_train_List=[]
        y_train_List = []

        for i in range(0, num_Interation):
            X_over, y_over = undersample.fit_resample(X_train, y_train)
            X_train_List.append(X_over.values)

            y_over = to_categorical(y_over)
            y_train_List.append(y_over)

        X_train_List = np.array(X_train_List)
        y_train_List = np.array(y_train_List)

        X_train_List = np.array(X_train_List)
        y_train_List = np.array(y_train_List)

        
        
        start = timeit.default_timer()
        tracemalloc.start()
        
        
        for i in range(0, num_Interation):
            training(X_train_List[i], y_train_List[i], i, epoch)

        current, peak = tracemalloc.get_traced_memory()
        stop = timeit.default_timer()
        Time_train=stop - start
        Train_Peak_RAM = peak / 10**6
        
        start = timeit.default_timer()
        tracemalloc.start()
        
        predicted = model_Predict(X_test)
        
        stop = timeit.default_timer()
        Time_predict=stop - start
        current, peak = tracemalloc.get_traced_memory()
        Peak_Time_prediction = peak / 10**6
        
        result = np.array([[0]*predicted[i].shape[1]]*predicted[i].shape[0])

        for i in range(0, num_Interation):
            result = result+ predicted[i]
        auc=roc_auc_score(y_test, result[:, 1]/num_Interation)
        
        thres = result.min()
        while (thres <= result.max()):
            thres = thres+result.max()/500
            yhat = []
            for i in range(len(result)):
                if result[i][0]>thres:
                    yhat.append(0)
                else:
                    yhat.append(1)

            yhat = np.array(yhat)

            tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(y_test, yhat)
            f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")


    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       

### 